<a href="https://colab.research.google.com/github/mohanrajmit/quality_inspection/blob/main/quality_inspection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### 1. Install TensorFlow Object Detection API
- For latest install instructions, check the [github](https://github.com/tensorflow/models/tree/master/research/object_detection) page or [readthedocs](https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/training.html) site for this API.

In [ ]:
#Clone github repository
!git clone https://github.com/tensorflow/models

Install TensorFlow Object Detection API

In [ ]:
%%bash
cd models/research
# Compile protos.
protoc object_detection/protos/*.proto --python_out=.
# Install TensorFlow Object Detection API.
cp object_detection/packages/tf2/setup.py .
python -m pip install --use-feature=2020-resolver .

In [ ]:
#Check if Object Detection API is installed
!pip list | grep object-detection

#### 2. Download dataset

Create a directory for the project (e.g detection) and move to project directory.

In [ ]:
#Come to the home directory - change if not running on  Colab
%cd /content

In [ ]:
#Create a folder for your project e.g in this case, detection. You can choose any name for the folder 
!mkdir detection

In [ ]:
#Go to the project folder
%cd detection

In [ ]:
!ls -l

In [ ]:
!git clone https://github.com/mohanrajmit/quality_inspection.git

In [ ]:
import pandas as pd
df_train = pd.read_csv('/content/detection/quality_inspection/train_labels.csv')
df_train.head(5)

In [ ]:
df_test = pd.read_csv('/content/detection/quality_inspection/test_labels.csv')
df_test.head(5)

In [ ]:
#How many object across all images
df_train.shape

In [ ]:
df_test.shape

In [ ]:
#List of Classes
df_train['class'].unique()

In [ ]:
#Number of labels
len(df_train['class'].unique())

We will need to Label encode classes e.g assign a unique index number for each class

In [ ]:
#Use Label encoder available in Scikit Learn
from sklearn import preprocessing

In [ ]:
#Label Encode class and add a 'label' column to the dataframe
le = preprocessing.LabelEncoder()
df_train['label'] = le.fit_transform(df_train['class'])
df_train.head()

In [ ]:
df_test['label'] = le.transform(df_test['class'])
df_test.head()

In [ ]:
#unique values in the label
df_train.label.unique()

In [ ]:
#unique values in the label
df_test.label.unique()

In [ ]:
#Object detection API expects index to start from 1 (and not 0)
df_train['label'] = df_train['label'] + 1
df_train.label.unique()

In [ ]:
#Object detection API expects index to start from 1 (and not 0)
df_test['label'] = df_test['label'] + 1
df_test.label.unique()

In [ ]:
#Dataframe should have label column now
df_train.head()

In [ ]:
df_test.head()

In [ ]:
#Create a dictionary of Label and Class. This will be useful for building our second input to Model training
label_class_dict = dict(zip(df_train['label'], df_train['class']))
print(label_class_dict)

**Visualizing the data**

In [ ]:
#We will use opencv and matplotlib
from matplotlib import pyplot as plt
import cv2
import numpy as np


In [ ]:
img_path = "/content/detection/quality_inspection/train"

In [ ]:
#Pickup a random image number
img_num = np.random.randint(0, df_train.shape[0])

#Read the image
img_file = df_train.loc[img_num,'filename']
img = cv2.imread(img_path + '/' + img_file)

#Find all rows which have same file name
rows_with_file = df_train[df_train['filename'] == img_file].index.tolist()

#Draw rectangle(s) as per bounding box information
print('Number of objects', len(rows_with_file))
for i in rows_with_file:

    #Get bounding box
    xmin, ymin, xmax, ymax = df_train.loc[i, ['xmin', 'ymin', 'xmax', 'ymax']]
    #Get Label
    label = df_train.loc[i, 'class']
    #Add bounding box
    cv2.rectangle(img, (xmin,ymin), (xmax, ymax), (0,255,0), 2)
    #Add text
    cv2.putText(img,label,(xmin, ymin+10),cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

#Convert BGR format (used by opencv to RGB format used by matplotlib)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

#cv2.imshow('img', img)

#Draw image using matplotlib
plt.figure(figsize=(10,7))
plt.imshow(img)
plt.show()

In [ ]:
df_train.groupby(['class']).count()

Save training and test data as csv

In [ ]:
df_train.to_csv('train.csv', index=False)
df_test.to_csv('test.csv', index=False)

The script file requires 3 inputs

1. --csv_input=<csv_file_path> : where is csv file located which was prepared in previous step
2. --img_path=<images_folder> : where are the actual images stored
3. --output_path=<output_file_path> : where the script can save the generated tfrecord file and what should be file name.

We will run script for training and test csv separately to create two tfrecord files.

In [ ]:
#generate tfrecord for training data
!python create_tfrecord.py --csv_input=train.csv  --img_path={img_path} --output_path=train.tfrecord

In [ ]:
test_img_path = "/content/detection/quality_inspection/test"

In [ ]:
#generate tfrecord for test data
!python generate_tfrecord.py --csv_input=test.csv  --img_path={test_img_path} --output_path=test.tfrecord

#### 5. Create Label Mapping File

In [ ]:
#Dict which was created earlier will be used for building Label Mapping file
label_class_dict

In [ ]:
#Build a pbtxt label file using label and class name
#This is required by Object detection API
#You can prepare it manually as well.

pbtxt_file_txt = ''
for label in sorted(label_class_dict.keys()):
    
    pbtxt_file_txt += "item {\n  id: " + str(label) + "\n  name: '" +  label_class_dict[label] + "'\n}\n\n"

with open('label_map.txt','w') as pbfile:
    pbfile.write(pbtxt_file_txt)

In [ ]:
!ls -l

In [ ]:
#Review the file content
!cat label_map.txt

#### 6. Download a pre-trained model

A list of pre-trained models is available at [TensorFlow model zoo](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf1_detection_zoo.md). We will use 'ssd_mobilenet_v1_coco' model for transfer learning.

In [ ]:
#Download the model from zoo
!wget -q http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_320x320_coco17_tpu-8.tar.gz

In [ ]:
#Extract tar file content
!tar -xf ssd_mobilenet_v2_320x320_coco17_tpu-8.tar.gz

#### 7. Prepare Training configuration file

1. Change num_classes parameter to 20 (as we have 20 categories in pascal voc dataset)
2. For 'train_input_reader' change 'input_path' to filepath of train.record file.
3. For 'train_input_reader' change 'label_map_path' to filepath of pascal_voc.pbtxt file.
4. Repeat above two steps for 'eval_input_reader'.
5. Change fine_tune_checkpoint to filepath where pre-trained model.ckpt file is available e.g ssd_mobilenet_v2_320x320_coco17_tpu-8/checkpoint/ckpt-0
6. Change 'batch_size' accordingly to available memory.
7. Change 'num_steps' to indicate how long the training will done e.g. 200000. For demo purpose, we are keeping it to 20 so that we can finish training quickly.


You can copy a sample configuration for the chosen pre-trained model (SSD MobileNet v2 320x320) in this case from [Configs](https://github.com/tensorflow/models/tree/master/research/object_detection/configs/tf2) folder. Here are things which need to be changed at a minimum.

Set Config file name

In [ ]:
config_file = 'ssd_mobilenet_v2_modified.config'

#### 8. Training the model

In [ ]:
#Copy training file from 'models/research/object_detection' folder to current folder
!cp /content/models/research/object_detection/model_main_tf2.py .

In [ ]:
!ls -l

Start training 

- Please note that Object detection take long time to train. The training may take few days if run on single GPU machine (depending on num of steps indicated). Try to keep training the model till loss comes close to 1 (or goes below 1). The script takes 3 inputs

1. --model_dir=<folder_name> : where model will be saved periodically as training progresses
2. --pipeline_config_path=<config_file_path> :where is model training configuration file located.

In [ ]:
#Create a training folder to store model checkpoints/snapshots as training progresses
!mkdir training

In [ ]:
#Check training folder
!ls -l training

In [ ]:
#start training
!python model_main_tf2.py --model_dir=training/ --pipeline_config_path={config_file} --checkpoint_every_n=100 --alsologtostderr

Model Evaluation

#### Training and Evaluation in Background

If we want to evaluate our model on training data regularly, we have to run both training and evaluation script in parallel. Model evaluation on test data gets done everytime model checkpoint is saved during training.

#### 9. Export trained model

From the saved model checkpoints, we will create a frozen trained model. Frozen here means to remove model nodes which are no longer needed in prediction. This reduces model size.

In [ ]:
#Copy export_inference_graph.py file from models/research/object_detection to current directory
!cp /content/models/research/object_detection/exporter_main_v2.py .

In [ ]:
!ls -l

The export_inference_graph.py script file requires the following input:

1. --input_type <input_node_name> : This will be used during prediction to set model input
2. --pipeline_config_path <model_training_config_file_path> : where is model training config file located.
3. --trained_checkpoint_prefix <file_path__model_checkpoint> : Which checkpoint should be used to create final model.
4. --output_directory <frozen_model_directory> : where should the frozen model created by script should be stored.

In [ ]:
#Provide input name, config file location, training folder
!python exporter_main_v2.py --input_type "image_tensor" --pipeline_config_path {config_file} --trained_checkpoint_dir training/ --output_directory detection_model

In [ ]:
label_class_dict

#### Model Prediction

In [ ]:
import tensorflow as tf

Load Saved model

In [ ]:
saved_model_path = 'detection_model/saved_model' #/gdrive/My Drive/AI-ML/models/pascal_voc_tf2/
model = tf.saved_model.load(saved_model_path)

Function to get model prediction

In [ ]:
#Function to get predictions from a Detection model
def detector_prediction(image_file, img_array=None, confidence_threshold=0.5):

    """
    image_file: File path of the image for which prediction needs to be done
    img_array: only considered if image_file is not specified
    confidence_threshold: Minimum confidence/probability for prediction to be considered
    """
    #Load image
    if(image_file):
        img = tf.keras.preprocessing.image.load_img(image_file)
        img_array = tf.keras.preprocessing.image.img_to_array(img).astype('uint8')
    
    #Make it a batch of one example
    img_array = tf.expand_dims(img_array, axis=0)

    #Prediction
    output = model(img_array) #get list of tensors discussed above as output
    
    #print(output)
    detection_classes = output['detection_classes'].numpy()[0]
    detection_scores = output['detection_scores'].numpy()[0] #get detection scores
    detection_boxes = output['detection_boxes'].numpy()[0]

    #Select predictions for which probability is higher than confidence_threshold
    selected_predictions = detection_scores >= confidence_threshold

    selected_prediction_scores = detection_scores[selected_predictions]
    selected_prediction_classes = detection_classes[selected_predictions]
    selected_prediction_boxes = detection_boxes[selected_predictions]

    #De-normalize box co-ordinates (multiply x-coordinates by image width and y-coords by image height)
    img_h, img_w = img_array.shape[1:3]

    for i in range(selected_prediction_boxes.shape[0]):
        
        selected_prediction_boxes[i,0] *= img_h #ymin * img_w
        selected_prediction_boxes[i,1] *= img_w #xmin * img_h
        selected_prediction_boxes[i,2] *= img_h #ymax * img_w
        selected_prediction_boxes[i,3] *= img_w #xmax * img_h

    #Make all co-ordinates as integer
    selected_prediction_boxes= selected_prediction_boxes.astype(int)

    #Convert class indexes to actual class labels
    predicted_classes = []
    for i in range(selected_prediction_classes.shape[0]):
        predicted_classes.append(label_class_dict[int(selected_prediction_classes[i])])

    #Number of predictions
    selected_num_predictions = selected_prediction_boxes.shape[0]

    return {'Total Predictions': selected_num_predictions,
            'Classes': predicted_classes, 
            'Scores': selected_prediction_scores, 
            'Box coordinates': selected_prediction_boxes}

In [ ]:
tf.keras.preprocessing.image.load_img('/content/detection/quality_inspection/test/fr1299.jpg')

In [ ]:
#Model output
detector_prediction('/content/detection/quality_inspection/test/fr1299.jpg', confidence_threshold=0.6)

Visualize model output

In [ ]:
import cv2
from matplotlib import pyplot as plt

In [ ]:
def visualize_output(image_file, confidence_threshold=0.5):

    #Call model prediction function above
    output = detector_prediction(image_file, confidence_threshold=confidence_threshold)

    #Read image
    img = cv2.imread(image_file)

    #Draw rectangle for predicted boxes, also add predicted classes
    for i in range(output['Box coordinates'].shape[0]):

        box = output['Box coordinates'][i]
        
        #Draw rectangle - (ymin, xmin, ymax, xmax)
        img = cv2.rectangle(img, (box[1], box[0]), (box[3], box[2]), (0,0,255), 3)
        
        #Add Label - Class name and confidence level
        label = output['Classes'][i] + ': ' + str(round(output['Scores'][i],2))
        img = cv2.putText(img, label, (box[1], box[0]-10), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
    
    #Conver BGR image to RGB to use with Matplotlib
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    #Display image
    plt.figure(figsize=(10,6))
    plt.imshow(img)
    plt.show()

In [ ]:
#Visualize on image
visualize_output('/content/detection/quality_inspection/test/fr1025.jpg', confidence_threshold=0.5)